In [6]:
"""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
from collections import OrderedDict, defaultdict
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import numpy as np
import pandas as pd
#from sqlalchemy import create_engine

# Create list of dictionaries for each row.
listings = []
imacscrapedir = '/Users/rayd/workspace/cinema/scrapes/'
airscrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-07-11.csv'
scrape_path = airscrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   
      
# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
#df = df[df.city != "Belleview, FL"]
#df = df[df.city != "High Springs, FL"]
#df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
#df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
#df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
# fix up titles, times and days, keep just needed columns, sort rows by movie+day
df.title = df.title.apply(lambda x: '"' + str(x) + '"')
df.times = df.times.str.replace(':00', '')
df.times = df.times.str.replace('|', ',')
df.times = df.times.str.replace('p', ' p.m.')
df.times = df.times.str.replace('a', ' a.m.')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('date', axis=1)
df['screen'] = df['title'].str.cat(df['format'],sep=", ")
df.screen = df.screen.str.cat(df['mpaa'],sep=", ")
df = df[['theater', 'screen', 'movieday', 'times']]
df.sort_values(["screen", "movieday"], axis =0,
              ascending=True, inplace=True)
df = df.drop_duplicates()

# Ocala market
oc6_df = df.loc[df['theater'].str.contains('Ocala Center 6')]
ocd_df = df.loc[df['theater'].str.contains('Drive-In')]
bv_df = df.loc[df['theater'].str.contains('Belleview')]
mt_df = df.loc[df['theater'].str.contains('Marion')]
omp_df = df.loc[df['theater'].str.contains('Old Mill')]
regaloca_df = df.loc[df['theater'].str.contains('Regal Hollywood')]
rialto_df = df.loc[df['theater'].str.contains('Rialto')]
amc_df = df.loc[df['theater'].str.contains('AMC Lake')]
barn_df = df.loc[df['theater'].str.contains('Barnstorm')]

# Gainesville market
celeb_df = df.loc[df['theater'].str.contains('Celebration')]
hipp_df = df.loc[df['theater'].str.contains('Hippodrome')]
butler_df = df.loc[df['theater'].str.contains('Butler')]
regalgnv_df = df.loc[df['theater'].str.contains('Regal Royal Park')]
uac_df = df.loc[df['theater'].str.contains('Regal UA Cinema 90')]
starke_df = df.loc[df['theater'].str.contains('Twin')]




In [7]:
omp_df

,theater,screen,movieday,times
216,Old Mill Playhouse,"""Hampstead"", Standard, PG-13",Friday,"9:35 a.m. , 2:20 p.m. , 4:30 p.m."
53,Old Mill Playhouse,"""Hampstead"", Standard, PG-13",Saturday,"9:35 a.m. , 2:20 p.m. , 4:30 p.m."
73,Old Mill Playhouse,"""Hampstead"", Standard, PG-13",Sunday,"9:35 a.m. , 2:20 p.m. , 4:30 p.m."
64,Old Mill Playhouse,"""Loopers: The Caddie's Long Walk"", Standard, PG",Friday,"12 p.m. , 4:45 p.m."
189,Old Mill Playhouse,"""Loopers: The Caddie's Long Walk"", Standard, PG",Saturday,"12 p.m. , 4:45 p.m."
106,Old Mill Playhouse,"""Loopers: The Caddie's Long Walk"", Standard, PG",Sunday,"12 p.m. , 4:45 p.m."
180,Old Mill Playhouse,"""Men in Black: International"", Standard, PG-13",Friday,"11:15 a.m. , 1:50 p.m. , 4:30 p.m. , 7:15 p.m."
118,Old Mill Playhouse,"""Men in Black: International"", Standard, PG-13",Saturday,"11:15 a.m. , 1:50 p.m. , 4:30 p.m. , 7:15 p.m."
127,Old Mill Playhouse,"""Men in Black: International"", Standard, PG-13",Sunday,"11:15 a.m. , 1:50 p.m. , 4:30 p.m. , 7:15 p.m."
293,Old Mill Playhouse,"""Phil (2019)"", Standard, R",Friday,"11:45 a.m. , 4:30 p.m."


In [ ]:
df.loc[df['theater'].str.contains('Rialto')]

In [ ]:
oc6_df = oc6_df.drop('theater', axis=1)
#oc6_df.set_index('screen')
#oc6_df.sort_values(["screen"], axis=0, 
#                 ascending=True, inplace=True)

In [ ]:
# Ocala market
oc6_df.to_csv('outfiles/oc6.csv', index=False, header=False) # Ocala 6
ocd_dr.to_csv('outfiles/ocd.csv', index=False, header=False) # Ocala Drive-in
bv_df.to_csv('outfiles/bv.csv', index=False, header=False) # Belleview
mt_df.to_csv('outfiles/mt.csv', index=False, header=False) # Marion
omp_df.to_csv('outfiles/omp.csv', index=False, header=False) # Old Mill Playhouse
regaloca_df.to_csv('outfiles/regaloca.csv', index=False, header=False) # Regal Hollywood
rialto_df.to_csv('outfiles/rialto.csv', index=False, header=False) # Rialto
amc_df.to_csv('outfiles/amc.csv', index=False, header=False) # AMC Lake Square 12
barn_df.to_csv('outfiles/barn.csv', index=False, header=False) # Barnstorm

# Gainesville market
celeb_df.to_csv('outfiles/celeb.csv', index=False, header=False) # Celebration Pointe
hipp_df.to_csv('outfiles/hipp.csv', index=False, header=False) # Hipp
butler_df.to_csv('outfiles/butler.csv', index=False, header=False) # Regal Butler Town Center
regalgnv_df.to_csv('outfiles/regalgnv.csv', index=False, header=False) # Regal Royal Park Stadium 17
uac_df.to_csv('outfiles/uac.csv', index=False, header=False) # UA Cinema
starke_df.to_csv('outfiles/starke.csv', index=False, header=False) # Florida Twin

In [ ]:
# Set index
#oc6_df.set_index('screen', 'movieday')
#oc6_df.set_index(['screen'], inplace=True)
oc6_df

In [ ]:
np.savetxt('oc6.txt', oc6_df.values, fmt='%s', delimiter=',', newline='\n', comments='#today')

In [ ]:
oc6_list = oc6_df.values.tolist()

In [ ]:
from itertools import groupby

for key, group in groupby(oc6_list, lambda x: x[0]):
    listOfThings = " and ".join([thing[1] for thing in group])
    print(key + ", " + listOfThings)

In [ ]:
i = 0
sizeofList = len(oc6_list) 
while i < sizeofList :
    if oc6_list[i][0] == oc6_list[i+1][0] and oc6_list[i][2] == oc6_list[i+1][2]:
        print(oc6_list[i][0] + ", " + oc6_list[i][1] + " at " + oc6_list[i][2]) 
    i += 1

In [ ]:
i = 0
sizeofList = len(oc6_list)
sizeofList = 6
while i < sizeofList :
    if oc6_list[i][1] == "Friday" and oc6_list[i+1][1] != "Saturday":
        print("It's a Friday only screening")
    elif oc6_list[i][1] == "Friday" and oc6_list[i+1][1] == "Saturday" and oc6_list[i+2][1] != "Sunday":
        print("It's a Friday-Saturday screening")
    elif oc6_list[i][1] == "Friday" and oc6_list[i][1] == "Saturday" and oc6_list[i][1] == "Sunday":
        print("We're here all weekend")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] != "Friday" and oc6_list[i+1]!= "Sunday":
        print("It's a Saturday only screening")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i+1][1] != "Friday" and oc6_list[i+2][1] == "Sunday":
        print("It's a Saturday-Sunday screening")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] == "Friday" and oc6_list[i][1] == "Sunday":
        print("We're here all weekend, but we're saying that again. Shall we drop this line?")
    elif oc6_list[i][1] == "Sunday" and oc6_list[i-1][1] != "Saturday":
        print("It's a Sunday only screening")
    elif oc6_list[i][1] == "Sunday" and oc6_list[i-2][1] != "Friday" and oc6_list[i-1][1] == "Saturday":
        print("It's a Saturday-Sunday screening, but we already said that. Shall we drop this line?")
        print("We're here all weekend, but we're saying that again. Shall we drop this line?")
    else:
        print("This isn't working for some reason")
    
    i += 1

In [ ]:
i = 0
while i < 18:
    if oc6_list[i][1] == "Friday" and oc6_list[i+1][1] == "Saturday" and oc6_list[i+2][1] == "Sunday":
        print("We're here all weekend")
    elif oc6_list[i][1] == "Friday" and oc6_list[i+1][1] == "Saturday" and oc6_list[i+2][1] != "Sunday":
        print("Looks like a Friday-Saturday screening.")
    elif oc6_list[i][1] == "Friday" and oc6_list[i+1][1] != "Saturday":
        print("Looks like a Friday-only screening.")
    if oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] == "Friday" and oc6_list[i+1][1] == "Sunday":
        print("We're here all weekend")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] == "Saturday" and oc6_list[i+1][1] != "Sunday":
        print("Looks like a Friday-Saturday screening, repeating.")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] != "Friday" and oc6_list[i][1] == "Sunday":
        print("Looks like a Saturday-Sunday screening.")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] != "Friday" and oc6_list[i+1][1] != "Sunday":
        print("Saturday only showing")
    else:
        print(i)
        
    i += 1

In [ ]:
i = 0
while i < 18:
    if oc6_list[i][] == "Friday" :
        fri = oc6_list[i][1]
    if oc6_list[i-1][1] == "Friday":
        lastfri = oc6_list[i-1][1]
    if oc6_list[i-1][1] == "Saturday":
        lastsat = oc6_list[i-1][1]
    if oc6_list[i-1][1] == "Sunday":
        lastsun = oc6_list[i-1][1]
    if oc6_list[i+1][1] == "Friday":
        nextfri = oc6_list[i+1][1]
    if oc6_list[i+1][1] == "Saturday":
        lastsat = oc6_list[i-1][1]
    if oc6_list[i+1][1] == "Sunday":
        lastsun = oc6_list[i-1][1]
    
    i += 1

In [ ]:
# KEEP THIS LOGIC

moviedict = {
    'Ocala6_AladinIMAX_Fri': {'times':'1, 2, 3'},
    'Ocala6_AladinIMAX_Sat': {'times':'1, 2, 3'},
    'Ocala6_AladinIMAX_Sun': {'times':'3, 2, 3'}
}
fri = str(moviedict['Ocala6_AladinIMAX_Fri']['times'])
sat = str(moviedict['Ocala6_AladinIMAX_Sat']['times'])
sun = str(moviedict['Ocala6_AladinIMAX_Sun']['times'])

if fri == sat and fri == sun: # want all common
    print("Friday-Sunday at " + fri) 
elif fri != sat and fri != sun and sat == sun: # want fri unique
    print("Friday at " + fri + ", Saturday-Sunday at " + sat)
elif sat != fri and sat != sun and fri == sun: # want sat unique
    print("Friday and Sunday at " + fri + ", Saturday at " + sat) 
elif sun != fri and sun != sat and fri == sat: # want sun unique
    print("Friday and Saturday at " + fri + ", Sunday at " + sun)
elif fri != sat and fri !=sun and sat !=sun: # want all unique
    print("Friday at " + fri + ", Saturday at " + sat + ",Sundaay at" + sun)
else:
    print("Show times didn't work out!")

In [ ]:
outfile = 'outfiles/oc6.csv'

with open(outfile) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    screen = []
    movieday = []
    times = []
    for row in readCSV:
        screen = row[0]
        movieday = row[1]
        times = row[2]
        
    print(screen)
    print(movieday)
    print(times)

In [ ]:
f = open(outfile, "r+")
ff = csv.reader(f)

pre_line = ff.next()
while(True):
    try:
        cur_line = ff.next()
        if pre_line == cur_line:
            pass #replace pass with do_something()
        pre_line = cur_line
    except:
        break

In [ ]:
outfile = 'outfiles/oc6.csv'

with open(outfile) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    screen = []
    movieday = []
    times = []
    for row in readCSV:
        screen = row[0]
        movieday = row[1]
        times = row[2]
        
    print(screen)
    print(movieday)
    print(times)

In [ ]:
# Ocala market
oc6_df = df.loc[df['theater'] == 'Ocala Center 6']
ocd_df = df.loc[df['theater'] == 'Ocala Drive-In']
bv_df = df.loc[df['theater'] == 'Belleview Cinemas']
mt_df = df.loc[df['theater'] == 'Marion Theatre']
omp_df = df.loc[df['theater'] == 'Old Mill Playhouse']
regaloca_df = df.loc[df['theater'] == 'Regal Hollywood Stadium 16 & IMAX - Ocala']
rialto_df = df.loc[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']
amc_df = df.loc[df['theater'] == 'AMC Lake Square 12']
barn_df = df.loc[df['theater'] == 'Barnstorm Theater']

# Gainesville market
celeb_df = df.loc[df['theater'] == 'Celebration Pointe 10']
hipp_df = df.loc[df['theater'] == 'Hippodrome State Theatre']
butler_df = df.loc[df['theater'] == 'Regal Butler Town Center 14']
regalgnv_df = df.loc[df['theater'] == 'Regal Royal Park Stadium 16']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']
starke_df = df.loc[df['theater'] == 'Florida Twin']

In [ ]:
ocala_market = [oc6_df, bv_df, mt_df, omp_df, regaloca_df, amc_df, barn_df, rialto_df, ocd_df]
gnv_market = [celeb_df, hipp_df, butler_df, regalgnv_df, uac_df, starke_df]

ocadf = pd.concat(ocala_market)
gnvdf = pd.concat(gnv_market)

In [ ]:
ocadf.to_csv('outfiles/oca.csv', index=False, header=False) # Ocala market
gnvdf.to_csv('outfiles/gnv.csv', index=False, header=False) # Ocala 6



In [ ]:
df[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']alto_df

In [ ]:
df[df['theater'].str.contains('Rialto')]

In [ ]:
df_t = pd.DataFrame(data = {
    'title' : ['AladinFri', 'AladinSat', 'AladinSun'],
    'time' : ['3p', '3p', '3p']
    })

In [ ]:
df_t

In [ ]:
from collections import OrderedDict, defaultdict
d = df_t.to_dict(into=OrderedDict)
d

In [ ]:
for i in items[1][1]:
    try:
        if items[1][1][i] == items[1][1][i + 1]:
            print("Friday at " + items[1][1][i])
        else:
            print("different time this day")
    except:
        pass

In [ ]:
if len(items[1][1]) == 3:
    for i in items[1][1]:
        fri = items[1][1][0]
        sat = items[1][1][1]
        sun = items[1][1][2]
    
        if fri == sat and fri == sun: # want all common
            print(items[0][1][1] + " Friday-Sunday at " + fri)
            break
        elif fri != sat and fri != sun and sat == sun: # want fri unique
            print("Friday at " + fri + ", Saturday-Sunday at " + sat)
        elif sat != fri and sat != sun and fri == sun: # want sat unique
            print("Friday and Sunday at " + fri + ", Saturday at " + sat) 
        elif sun != fri and sun != sat and fri == sat: # want sun unique
            print("Friday and Saturday at " + fri + ", Sunday at " + sun)
        elif fri != sat and fri !=sun and sat !=sun: # want all unique
            print("Friday at " + fri + ", Saturday at " + sat + ",Sundaay at" + sun)
        else:
            print("Show times didn't work out!")

In [ ]:
od_celeb = celeb_df.to_dict(into=OrderedDict)

In [ ]:
od_celeb